# Azure Custom Vision Parking Lot Demo


In [ ]:
# Replace with valid keys

training_key = "xxxx"
prediction_key = "xxxxx"


In [ ]:
!pip install azure-cognitiveservices-vision-customvision
!pip install urllib

In [ ]:
import os
import json
import time
import glob

from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region

trainer = training_api.TrainingApi(training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")

# Create a new project
print ("Creating project...")
project = trainer.create_project(
    "Parking Lot Demo", domain_id=obj_detection_domain.id)

# Make two tags in the new project
car_tag = trainer.create_tag(project.id, "car")

# Load training json and add it to the training set
with open('training_images/train.json') as f:
    training_data = json.load(f)

print ("Adding images...")
tagged_images_with_regions = []
for t in training_data:
    regions = []
    for r in t["regions"]:
        regions.append(Region(tag_id=car_tag.id, left=r["x"], top=r["y"], width=r["w"], height=r["h"]))
      
    with open("training_images/" + t["file"], mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=t["file"], contents=image_contents.read(), regions=regions))


trainer.create_images_from_files(project.id, images=tagged_images_with_regions)

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")
    

In [ ]:
import glob
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from PIL import Image

from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

for f in glob.glob("test_images/*.jpg"):
    with open(f, mode="rb") as test_data:
        results = predictor.predict_image(project.id, test_data, iteration.id)
        
        plt.figure(figsize=(5, 5))
        image = Image.open(test_data)
        ax = plt.imshow(image, alpha=0.5)
        plt.text(0, -10, f, fontsize=12, va="top")

        for p in results.predictions:
            if p.probability < 0.5:
                continue

            x = (p.bounding_box.left + p.bounding_box.width/2) * image.width
            y = (p.bounding_box.top + p.bounding_box.height/2) * image.height
            text = p.tag_name + " {0:.2f}%".format(p.probability * 100)
            patch  = Circle((x, y), 10, fill=False, linewidth=2, color='y')
            ax.axes.add_patch(patch)
            plt.text(x, y, text, fontsize=10, va="top")
        plt.axis("off")
